In [6]:
%%writefile practice71.cu

// Записываем код  в файл для компиляции CUDA
// Practice 7 Task 1
// 1. Напишите ядро CUDA для выполнения редукции (суммирования элементов массива).
// 2. Используйте разделяемую память для оптимизации доступа к данным.
// 3. Проверьте корректность работы на тестовом массиве.


#include <iostream>               // Для стандартный ввод-вывод
#include <cuda_runtime.h>         // Для CUDA Runtime API
#include <chrono>                 // Для измерения времени

using namespace std;              // Чтобы не писать std::


// CUDA ЯДРО: ПАРАЛЛЕЛЬНАЯ РЕДУКЦИЯ
__global__ void reductionKernel(int *d_input, int *d_output, int n) {
    extern __shared__ int sdata[];                              // Объявляем разделяемую память внутри блока

    unsigned int tid = threadIdx.x;                             // tid — локальный индекс потока внутри блока
    unsigned int i = blockIdx.x * blockDim.x + tid;             // i — глобальный индекс элемента массива

    if (i < n)                                                  // Проверяем, не вышел ли поток за границы массива
        sdata[tid] = d_input[i];                                // Загружаем элемент из глобальной памяти в shared memory
    else
        sdata[tid] = 0;                                         // Если вышли за границу — записываем 0

    __syncthreads();                                            // Синхронизируем все потоки блока перед редукцией

    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {     // На каждом шаге уменьшаем активные потоки в 2 раза
        if (tid < s)                                            // Работает только первая половина потоков
            sdata[tid] += sdata[tid + s];                       // Складываем пару элементов в shared памяти
        __syncthreads();                                        // Синхронизация перед следующим шагом
    }

    if (tid == 0)                                               // Только нулевой поток блока
        d_output[blockIdx.x] = sdata[0];                        // Записывает сумму всего блока в глобальную память
}

// ФУНКЦИЯ ЗАПУСКА ОДНОГО ТЕСТА
void runTest(int N) {
    cout << "\n Размер массива: " << N  << endl;// Выводим текущий размер тестируемого массива

    int SIZE = N * sizeof(int);                                 // Вычисляем размер массива в байтах

    int *h_array = new int[N];                                  // Выделяем память под массив в оперативной памяти (CPU)

    for (int i = 0; i < N; i++)                                 // Заполняем массив значениями
        h_array[i] = 1;                                         // Присваиваем всем элементам значение 1

    // ПОСЛЕДОВАТЕЛЬНАЯ РЕДУКЦИЯ НА CPU
    int cpu_sum = 0;                                            // Переменная для хранения суммы на CPU

    auto cpu_start = chrono::high_resolution_clock::now();     // Запускаем таймер CPU

    for (int i = 0; i < N; i++)                                 // Последовательный цикл суммирования
        cpu_sum += h_array[i];                                  // Прибавляем каждый элемент массива

    auto cpu_end = chrono::high_resolution_clock::now();       // Останавливаем таймер CPU

    chrono::duration<double> cpu_time = cpu_end - cpu_start;   // Вычисляем время выполнения CPU

    // ПАРАЛЛЕЛЬНАЯ РЕДУКЦИЯ НА GPU
    int *d_input, *d_intermediate;                              // Указатели на память GPU

    cudaMalloc(&d_input, SIZE);                                 // Выделяем память на GPU под входной массив

    cudaMemcpy(d_input, h_array, SIZE, cudaMemcpyHostToDevice);// Копируем данные CPU → GPU

    int threadsPerBlock = 256;                                  // Количество потоков в одном блоке

    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
                                                                 // Вычисляем количество блоков в сетке

    cudaMalloc(&d_intermediate, blocksPerGrid * sizeof(int));  // Память под промежуточные суммы блоков

    cudaEvent_t gpu_start, gpu_stop;                            // Объявляем CUDA-события для измерения времени GPU

    cudaEventCreate(&gpu_start);                                // Создаём событие старта
    cudaEventCreate(&gpu_stop);                                 // Создаём событие окончания

    cudaEventRecord(gpu_start);                                 // Запускаем таймер GPU

    reductionKernel<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(
        d_input, d_intermediate, N);                            // Запускаем CUDA-ядро редукции

    int *h_intermediate = new int[blocksPerGrid];              // Выделяем массив CPU для хранения сумм блоков

    cudaMemcpy(h_intermediate, d_intermediate,
               blocksPerGrid * sizeof(int), cudaMemcpyDeviceToHost);
                                                                 // Копируем суммы блоков GPU → CPU

    int gpu_sum = 0;                                            // Переменная для итоговой суммы GPU

    for (int i = 0; i < blocksPerGrid; i++)                     // Последовательное суммирование сумм блоков
        gpu_sum += h_intermediate[i];                           // Финальная редукция на CPU

    cudaEventRecord(gpu_stop);                                  // Останавливаем таймер GPU
    cudaEventSynchronize(gpu_stop);                             // Дожидаемся завершения события

    float gpu_time = 0;                                         // Переменная для хранения времени GPU
    cudaEventElapsedTime(&gpu_time, gpu_start, gpu_stop);       // Получаем время выполнения GPU в миллисекундах

    // Вывод результатов
    cout << "CPU результат: " << cpu_sum << endl;                  // Вывод суммы, полученной на CPU
    cout << "GPU результат: " << gpu_sum << endl;                  // Вывод суммы, полученной на GPU
    cout << "CPU время: " << cpu_time.count() * 1000 << " мс" << endl; // Вывод времени CPU
    cout << "GPU время: " << gpu_time << " мс" << endl;          // Вывод времени GPU

    // Освобождение памяти
    delete[] h_array;                                           // Освобождаем память CPU массива
    delete[] h_intermediate;                                    // Освобождаем промежуточный массив CPU
    cudaFree(d_input);                                          // Освобождаем память GPU входного массива
    cudaFree(d_intermediate);                                   // Освобождаем память GPU промежуточных сумм
}

// Основная функция
int main() {
    runTest(1024);                                              // Тестируем массив из 1024 элементов
    runTest(1000000);                                           // Тестируем массив из 1 000 000 элементов
    runTest(10000000);                                          // Тестируем массив из 10 000 000 элементов
    return 0;                                                   // Завершаем программу
}


Overwriting practice71.cu


In [14]:
# Компиляция
!nvcc practice71.cu -o practice71 -arch=sm_75 -std=c++11            # -arch=sm_75  - архитектура GPU (Tesla T4 в Colab = sm_75)
                                                                    # -std=c++11 — стандарт C++
# Запуск
!./practice71



 Размер массива: 1024
CPU результат: 1024
GPU результат: 1024
CPU время: 0.0027 мс
GPU время: 0.10752 мс

 Размер массива: 1000000
CPU результат: 1000000
GPU результат: 1000000
CPU время: 2.56092 мс
GPU время: 0.14976 мс

 Размер массива: 10000000
CPU результат: 10000000
GPU результат: 10000000
CPU время: 25.9017 мс
GPU время: 1.21094 мс
